In [ ]:
import sys
import heapq
from collections import defaultdict


def levenshtein(s1, s2):
    s1 = [ord(x) for x in s1]
    s2 = [ord(x) for x in s2]

    VP = (1 << len(s1)) - 1
    VN = 0
    currDist = len(s1)
    mask = 1 << (len(s1) - 1)

    block = {}
    block_get = block.get
    x = 1
    for ch1 in s1:
        block[ch1] = block_get(ch1, 0) | x
        x <<= 1

    for ch2 in s2:
        PM_j = block_get(ch2, 0)
        X = PM_j
        D0 = (((X & VP) + VP) ^ VP) | X | VN
        HP = VN | ~(D0 | VP)
        HN = D0 & VP
        currDist += (HP & mask) != 0
        currDist -= (HN & mask) != 0
        HP = (HP << 1) | 1
        HN = HN << 1
        VP = HN | ~(D0 | HP)
        VN = HP & D0
    return currDist


def create_trigrams(s):
    s = ' ' + s + ' '
    return [s[i:i + 3] for i in range(len(s) - 2)]


def create_trigram_index(books):
    trigram_index = defaultdict(list)
    for book in books:
        trigrams = create_trigrams(book)
        for trigram in trigrams:
            trigram_index[trigram].append(book)
    return trigram_index


def find_candidates(query, trigram_index):
    query_trigrams = create_trigrams(query)
    candidate_counts = defaultdict(int)
    for trigram in query_trigrams:
        if trigram in trigram_index:
            for book in trigram_index[trigram]:
                candidate_counts[book] += 1
    sorted_candidates = sorted(candidate_counts, key=candidate_counts.get, reverse=True)
    return sorted_candidates[:10]


def process_queries(books, queries):
    trigram_index = create_trigram_index(books)
    results = []
    for query in queries:
        candidates = find_candidates(query, trigram_index)
        top_candidates = []
        for candidate in candidates:
            distance = levenshtein(query, candidate)
            heapq.heappush(top_candidates, (distance, candidate))
        best_matches = [heapq.heappop(top_candidates)[1] for _ in range(min(3, len(top_candidates)))]
        results.append(best_matches)
    return results
    

def main():
    input = sys.stdin.read
    data = input().split('\n')
    
    n = int(data[0].strip())
    book_titles = [data[i + 1].strip() for i in range(n)]
    m = int(data[n + 1].strip())
    queries = [data[n + 2 + i].strip() for i in range(m)]

    result = process_queries(book_titles, queries)
    for book_list in result:
        if len(book_list) == 0:
            print(1)
            print(book_titles[0])
        else:
            print(len(book_list))
            for book in book_list:
                print(book)


if __name__ == '__main__':
    main()